In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps
from osgeo import gdal

In [2]:
lat, lon = 32.689670,109.004300

In [3]:
m = leafmap.Map(draw_control=True, layers_control=True, measure_control=True, fullscreen_control=False, attribution_control=True)
m.add_basemap("SATELLITE")
m.set_center(lon,lat,16)
m

Map(center=[32.68967, 109.0043], controls=(AttributionControl(options=['position', 'prefix'], position='bottom…

In [ ]:
m.user_roi_bounds

In [ ]:
def download_geotiff(Name,Zoom,geotiff_path,source):
    tms_to_geotiff(output=geotiff_path,bbox=m.user_roi_bounds(), zoom=Zoom,source=source,overwrite=True)
def cal_epsg():
    center = round(m.center[1],0)
    epsg = int(4548 - round((117-center)/3,0))
    print(f'本项目中央子午线精度为：{center}，投影坐标系应为 EPSG:{epsg}')
    return epsg
def reproject_geotiff(src_filename, dst_filename, dst_srs):
    # Open the source file
    src_ds = gdal.Open(src_filename)
    if src_ds is None:
        print('Could not open source file', src_filename)
        return
    
    # Fetch the source SRS
    src_srs = src_ds.GetProjectionRef()
    
    # Create the destination file
    dst_ds = gdal.Warp(dst_filename, src_ds, dstSRS=dst_srs)
    print(src_filename,dst_filename,dst_srs)

    # Close datasets
    if dst_ds:
        dst_ds = None
    if src_ds:
        src_ds = None

def generate_tfw(geotiff_path, tfw_path):
    """Generate a TFW file from a GeoTIFF file.

    Args:
        geotiff_path: The path to the GeoTIFF file.
        tfw_path: The path to the TFW file.
    """

    dataset = gdal.Open(Re_geotiff_path)

    if dataset is None:
        raise Exception("Could not open GeoTIFF file.")

    transform = dataset.GetGeoTransform()
    transform = [transform[1], transform[2], transform[4], transform[5], transform[0], transform[3]]

    with open(tfw_path, "w") as f:
        f.write("\n".join([f"{x:.16f} " for x in transform]))

In [ ]:
if __name__ == "__main__":
    image_folder = r'D:\Documents\WeChat Files\wxid_3oemgw81j4rc22\FileStorage\File\2023-10\IMG_20231030_162539(1)'
    epsg = cal_epsg()
    Name = "安康项目"
    source = "SATELLITE"
    temp  = "D:\Temp"
    os.chdir(temp)
    # base_path = r"W:\20230808 腾冲火山地质公园\08-现场照片\卫片"
    # os.chdir(base_path)
    os.makedirs(f"./{Name}", exist_ok=True)
    Zoom = 21
    geotiff_path = f"./{Name}/{Name}_{source}_4326.tif"
    Re_geotiff_path = f"./{Name}/{Name}_{source}_{epsg}.tif"
    download_geotiff(Name,Zoom,geotiff_path,source)
    reproject_geotiff(geotiff_path, Re_geotiff_path, f'EPSG:{epsg}')
    tfw_path = f"./{Name}/{Name}_{source}_{epsg}.tfw"
    generate_tfw(geotiff_path, tfw_path)
    os.system('start .')